In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

def get_variations_list(content):
    # 获取典故的 allusion_block_list 的列表
    allusion_block_list = content.find_all(name="div", attrs={"class": "allusionBlock"})

    variation_list = []

    for child in allusion_block_list[0]:
        if isinstance(child, str):  # 跳过文本节点
            continue
        previous_siblings = child.find_previous_siblings()

        # print(f"当前元素: {child.name} - {child.text}")
        # print("前面的兄弟元素:")
        flag = True

        # 如果元素含有 “典故” 、 “相关人物” 或 “参考典故” ，那么不要这个元素
        if((child.get_text()=="典故") | (child.get_text()=="相关人物") | (child.get_text()=="参考典故") | (child.get_text()=="")):
            flag = False

        for sibling in previous_siblings:
            # print(f"  {sibling.name} - {sibling.text}")
            # 如果前面的元素有 “相关人物” 和 “参考典故” ，那么不要这个元素
            if((sibling.get_text()=="相关人物") | (sibling.get_text()=="参考典故")):
                flag = False
        if(flag):
            variation_list.append(child.get_text())

        #print('-' * 50)
    # print(variation_list)
    # print(len(variation_list))
    return variation_list

def get_poem_list(content):
    # 获取典故的 allusion_block_list 的列表
    allusion_block_list = content.find_all(name="div", attrs={"class": "allusionBlock"})

    poem_list = []

    child_list = allusion_block_list[-1].find_all(name="p")
    for child in child_list:
        poem_dict = {}
        # print(child.contents[0])
        sentence = child.contents[0].strip()
        if(child.find(name="span")):
            # print(child.find(name="span").contents[0])
            author = child.find(name="span").contents[0].strip()
            if(child.find(name="a")):
                title = child.find(name="span").contents[1].get_text().strip()
                poem_dict["sentence"] = sentence
                poem_dict["author"] = author
                poem_dict["title"] = title
                poem_list.append(poem_dict)
            else:
                poem_dict["sentence"] = sentence
                poem_dict["author"] = author.split()[0].strip()
                poem_dict["title"] = author.split()[1].strip()
                poem_list.append(poem_dict)
    # print(poem_list)
    # print(len(poem_list))
    return poem_list

def filter_contained_lists(lst):
    result = []
    for i, sublist1 in enumerate(lst):
        is_contained = False
        for j, sublist2 in enumerate(lst):
            if i != j and set(sublist1).issubset(set(sublist2)):
                is_contained = True
                break
        if not is_contained:
            result.append(sublist1)
    return result

def get_added_poem_list(allusion, variation_list, poem_list):
    # 遍历这个典故的所有诗句
    for poem_dict in poem_list:

        # 设置该诗句的典故
        poem_dict["allusion"] = allusion

        # 初始化该诗句涉及到的典故的异形的数量为 0
        poem_dict["variation_number"] = 0

        # 初始化该诗句的典故异性的位置列表
        poem_dict["allusion_index"] = []

        # 遍历这个典故的异形列表
        for variation in variation_list:
            # 如果该诗句包含本异形 variation
            if(variation in poem_dict["sentence"]):
                # 将该异形的位置加到 index列表里面
                poem_dict["allusion_index"].append(list(range(poem_dict["sentence"].find(variation), poem_dict["sentence"].find(variation) + len(variation))))
                poem_dict["variation_number"]+=1
        # if(poem_dict["variation_number"]==0):
        #     print(str(poem_dict) + "\t" + "出现典故大于一次")
        poem_dict['allusion_index'] = filter_contained_lists(poem_dict['allusion_index'])
    return poem_list

def process_one_page(url_of_page,url_of_file):
    # 获取HTML文本
    res = requests.get(
        url_of_page
    )

    soup = BeautifulSoup(res.text, features="html.parser")

    # 获取典故名字列表
    allusion_list = [x.get_text() for x in soup.find(attrs={"id": "IndexPanel"}).find_all(name="a")]

    # 获取 ContentPanel 列表
    content_list = soup.find(attrs={"id": "ContentPanel"}).find_all(attrs={"class": "allusion"})

    if_have_all_poem_df = False

    # 遍历 allusion_list 和 content_list ，即遍历这一页的所有典故
    for allusion, content in zip(allusion_list, content_list):

        # 获取该典故的异形的列表
        variation_list = get_variations_list(content)

        # 获取该典故的所有诗句的列表，列表元素是字典
        poem_list = get_poem_list(content)
        
        # 对该典故的所有诗句里面的词的位置进行标注
        poem_list = get_added_poem_list(allusion, variation_list, poem_list)
        
        temp_df = pd.DataFrame(poem_list)
    
        if(if_have_all_poem_df==False):
            if_have_all_poem_df = True
            all_poem_df = temp_df
        else:
            all_poem_df = pd.concat([all_poem_df,temp_df], ignore_index=True)# ignore_index 参数用于重新设置索引
    # print(all_poem_df[["allusion_index"]].head())
    # 检查文件是否存在
    if os.path.exists(url_of_file):
        # 如果文件存在，追加数据时不保存列名
        all_poem_df.to_csv(url_of_file, sep='\t', mode='a', index=False, header=False)
    else:
        # 如果文件不存在，保存数据并保存列名
        all_poem_df.to_csv(url_of_file, sep='\t', index=False)
    return all_poem_df.shape[0]

if __name__ == "__main__":
    all_len = 0
    for i in range(0,102,1):
        url_of_page = "https://sou-yun.cn/AllusionsIndex.aspx?page=" + str(i)
        url_of_file = "D:/软件包/爬取的典故数据.csv"
        this_page_len = process_one_page(url_of_page,url_of_file)
        all_len+=this_page_len
        print("爬完了第" + str(i+1) + "页的数据，本页共有" + str(this_page_len) + "条数据")
    print("共有" + str(all_len) + "条数据")

爬完了第1页的数据，本页共有1207条数据
爬完了第2页的数据，本页共有707条数据
爬完了第3页的数据，本页共有593条数据
爬完了第4页的数据，本页共有491条数据
爬完了第5页的数据，本页共有446条数据
爬完了第6页的数据，本页共有420条数据
爬完了第7页的数据，本页共有390条数据
爬完了第8页的数据，本页共有364条数据
爬完了第9页的数据，本页共有338条数据
爬完了第10页的数据，本页共有324条数据
爬完了第11页的数据，本页共有299条数据
爬完了第12页的数据，本页共有283条数据
爬完了第13页的数据，本页共有273条数据
爬完了第14页的数据，本页共有258条数据
爬完了第15页的数据，本页共有242条数据
爬完了第16页的数据，本页共有233条数据
爬完了第17页的数据，本页共有222条数据
爬完了第18页的数据，本页共有211条数据
爬完了第19页的数据，本页共有204条数据
爬完了第20页的数据，本页共有198条数据
爬完了第21页的数据，本页共有190条数据
爬完了第22页的数据，本页共有186条数据
爬完了第23页的数据，本页共有180条数据
爬完了第24页的数据，本页共有177条数据
爬完了第25页的数据，本页共有170条数据
爬完了第26页的数据，本页共有167条数据
爬完了第27页的数据，本页共有160条数据
爬完了第28页的数据，本页共有159条数据
爬完了第29页的数据，本页共有150条数据
爬完了第30页的数据，本页共有150条数据
爬完了第31页的数据，本页共有146条数据
爬完了第32页的数据，本页共有140条数据
爬完了第33页的数据，本页共有140条数据
爬完了第34页的数据，本页共有132条数据
爬完了第35页的数据，本页共有130条数据
爬完了第36页的数据，本页共有127条数据
爬完了第37页的数据，本页共有120条数据
爬完了第38页的数据，本页共有120条数据
爬完了第39页的数据，本页共有120条数据
爬完了第40页的数据，本页共有115条数据
爬完了第41页的数据，本页共有110条数据
爬完了第42页的数据，本页共有110条数据
爬完了第43页的数据，本页共有110条数据
爬完了第44页的数据，本页共有108条数据
爬完了第45页的数据，本页共有100条数据
爬完了第46页的数据，本页共有100

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

def get_variations_list(content):
    # 获取典故的 allusion_block_list 的列表
    allusion_block_list = content.find_all(name="div", attrs={"class": "allusionBlock"})

    variation_list = []

    for child in allusion_block_list[0]:
        if isinstance(child, str):  # 跳过文本节点
            continue
        previous_siblings = child.find_previous_siblings()

        # print(f"当前元素: {child.name} - {child.text}")
        # print("前面的兄弟元素:")
        flag = True

        # 如果元素含有 “典故” 、 “相关人物” 或 “参考典故” ，那么不要这个元素
        if((child.get_text()=="典故") | (child.get_text()=="相关人物") | (child.get_text()=="参考典故") | (child.get_text()=="")):
            flag = False

        for sibling in previous_siblings:
            # print(f"  {sibling.name} - {sibling.text}")
            # 如果前面的元素有 “相关人物” 和 “参考典故” ，那么不要这个元素
            if((sibling.get_text()=="相关人物") | (sibling.get_text()=="参考典故")):
                flag = False
        if(flag):
            variation_list.append(child.get_text())

        #print('-' * 50)
    # print(variation_list)
    # print(len(variation_list))
    return variation_list
def process_one_page(url_of_page,url_of_file):
    # 获取HTML文本
    res = requests.get(
        url_of_page
    )

    soup = BeautifulSoup(res.text, features="html.parser")

    # 获取典故名字列表
    allusion_list = [x.get_text() for x in soup.find(attrs={"id": "IndexPanel"}).find_all(name="a")]

    # 获取 ContentPanel 列表
    content_list = soup.find(attrs={"id": "ContentPanel"}).find_all(attrs={"class": "allusion"})

    this_page_list = []

    # 遍历 allusion_list 和 content_list ，即遍历这一页的所有典故
    for allusion, content in zip(allusion_list, content_list):

        # 获取该典故的异形的列表
        variation_list = get_variations_list(content)

        # 获取一个典故的字典
        allusion_dict = {}
        allusion_dict["allusion"] = allusion
        allusion_dict["variation_list"] = variation_list
        
        this_page_list.append(allusion_dict)
    this_page_df = pd.DataFrame(this_page_list)
    # 检查文件是否存在
    if os.path.exists(url_of_file):
        # 如果文件存在，追加数据时不保存列名
        this_page_df.to_csv(url_of_file, sep='\t', mode='a', index=False, header=False)
    else:
        # 如果文件不存在，保存数据并保存列名
        this_page_df.to_csv(url_of_file, sep='\t', index=False)
    return this_page_df.shape[0]

if __name__ == "__main__":
    all_len = 0
    for i in range(0,102,1):
        url_of_page = "https://sou-yun.cn/AllusionsIndex.aspx?page=" + str(i)
        url_of_file = "D:/软件包/典故的异形数据.csv"
        this_page_len = process_one_page(url_of_page,url_of_file)
        all_len+=this_page_len
        print("爬完了第" + str(i+1) + "页的数据，本页共有" + str(this_page_len) + "条数据")
    print("共有" + str(all_len) + "条数据")

爬完了第1页的数据，本页共有10条数据
爬完了第2页的数据，本页共有10条数据
爬完了第3页的数据，本页共有10条数据
爬完了第4页的数据，本页共有10条数据
爬完了第5页的数据，本页共有10条数据
爬完了第6页的数据，本页共有10条数据
爬完了第7页的数据，本页共有10条数据
爬完了第8页的数据，本页共有10条数据
爬完了第9页的数据，本页共有10条数据
爬完了第10页的数据，本页共有10条数据
爬完了第11页的数据，本页共有10条数据
爬完了第12页的数据，本页共有10条数据
爬完了第13页的数据，本页共有10条数据
爬完了第14页的数据，本页共有10条数据
爬完了第15页的数据，本页共有10条数据
爬完了第16页的数据，本页共有10条数据
爬完了第17页的数据，本页共有10条数据
爬完了第18页的数据，本页共有10条数据
爬完了第19页的数据，本页共有10条数据
爬完了第20页的数据，本页共有10条数据
爬完了第21页的数据，本页共有10条数据
爬完了第22页的数据，本页共有10条数据
爬完了第23页的数据，本页共有10条数据
爬完了第24页的数据，本页共有10条数据
爬完了第25页的数据，本页共有10条数据
爬完了第26页的数据，本页共有10条数据
爬完了第27页的数据，本页共有10条数据
爬完了第28页的数据，本页共有10条数据
爬完了第29页的数据，本页共有10条数据
爬完了第30页的数据，本页共有10条数据
爬完了第31页的数据，本页共有10条数据
爬完了第32页的数据，本页共有10条数据
爬完了第33页的数据，本页共有10条数据
爬完了第34页的数据，本页共有10条数据
爬完了第35页的数据，本页共有10条数据
爬完了第36页的数据，本页共有10条数据
爬完了第37页的数据，本页共有10条数据
爬完了第38页的数据，本页共有10条数据
爬完了第39页的数据，本页共有10条数据
爬完了第40页的数据，本页共有10条数据
爬完了第41页的数据，本页共有10条数据
爬完了第42页的数据，本页共有10条数据
爬完了第43页的数据，本页共有10条数据
爬完了第44页的数据，本页共有10条数据
爬完了第45页的数据，本页共有10条数据
爬完了第46页的数据，本页共有10条数据
爬完了第47页的数据，本页共有10条数据
爬完了第48页的数据，本页共有10条数据
爬